#### 先导入实验所需的包或模块

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import sys,os
import_path=!cd ..&&cd
sys.path.append(import_path[0])

from mxnet import gluon, image, init, nd, gpu
from mxnet.gluon import data as gdata, loss as gloss, model_zoo, nn
import numpy as np
import d2lzh as d2l
import cloudpickle as pkl

print("导入模块成功")

导入模块成功


#### 反序列化训练好的网络

In [2]:
net_filename = "hand_segmentation_net.pkl"
net = None
net_filepath = os.path.join(d2l.ROOT_DIR, "model",net_filename)
with open(net_filepath,"rb") as net_f:
    net = pkl.load(net_f)
print("Read net successfully")

Read net successfully


#### 定义测试函数与映射函数

In [3]:
def predict(img):
    X = test_iter._dataset.normalize_image(img)
    X = X.transpose((2, 0, 1)).expand_dims(axis=0)
    pred = nd.argmax(net(X.as_in_context(ctx[0])), axis=1)
    return pred.reshape((pred.shape[1], pred.shape[2]))

# 为了可视化每个像素的预测类别，我们将预测类别映射回它们在数据集中的标注颜色。
def label2image(pred):
    colormap = nd.array(d2l.hand_colormap, ctx=ctx[0], dtype='uint8')
    X = pred.astype('int32')
    return colormap[X, :]

#### 导入数据及定义裁剪尺寸

In [12]:
# 文件目录
feature_dir = "1_train_rgb"
label_dir = "2_train_mask"
test_dir = "3_test_rgb"
result_dir="test_mask"#你的文件目录
result_dir= os.path.join(d2l.ROOT_DIR, result_dir) 
pictures=os.listdir(os.path.join(d2l.ROOT_DIR,"data",test_dir))
if pictures == []:
    raise Exception("没有正确的放置图片的位置")

# 定义实验参数
crop_size, batch_size, colormap2label = (320, 480), 32, nd.zeros(256**3)
num_workers = 0 if sys.platform.startswith('win32') else 4
ctx = d2l.try_all_gpus()

#测试集
test_iter = gdata.DataLoader(
    d2l.HandDataset(feature_dir, label_dir, crop_size, colormap2label,1000,7000), batch_size,
    last_batch='discard', num_workers=num_workers)

开始读取图像...
当前文件夹为:D:\computer vision\data\1_train_rgb
照片总数为:0
读取完毕
开始读取图像...
当前文件夹为:D:\computer vision\data\2_train_mask
照片总数为:0
读取完毕
read 0 examples


#### 预测出手部分割结果 即掩码图

In [ ]:
num_picture = 10 #图片数量
index_picture = 0 #图片索引
test_images2, images_name= d2l.read_images(test_dir, num_picture, index_picture)
crop_rect = (0, 0, 480, 320)
images=[]
for i in range(num_picture):
    pred = label2image(predict(test_images2[i]))
    images += [test_images2[i], pred]
    plt.imsave(os.path.join(result_dir,images_name[i]),pred.asnumpy(),cmap="gray")
 
print("图片已保存到test_mask")
d2l.show_images(images[::2]+images[1::2],2,10)